In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

/home/ljj0512/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


## check the total number of genres

In [2]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT genres FROM Korean_book")
check = cur.fetchall()
print(check[0:3])
conn.commit()
conn.close()


[('국내도서,소설,한국소설,역사/대하소설,장르소설,역사소설',), ('국내도서,소설,한국소설,한국소설일반',), ('국내도서,소설,한국소설,한국소설일반',)]


In [3]:
rdd = sc.parallelize(check).map(lambda line: line[0].split(","))
print(rdd.count())
rdd = rdd.flatMap(lambda x: x).map(lambda x: (x,1))
print(rdd.take(5))
print(rdd.count())


44749
[('국내도서', 1), ('소설', 1), ('한국소설', 1), ('역사/대하소설', 1), ('장르소설', 1)]
244815


In [12]:
rdd1 = rdd.reduceByKey(lambda x, y: x+y).sortBy(lambda x: -x[1]) # descending
rdd2 = rdd.reduceByKey(lambda x, y: x+y).sortBy(lambda x: x[1]) # ascending
print("total genres:",rdd1.count()-1) # <-- (-1)은 국내도서를 없애기 위해

total genres: 2860


In [13]:
top = 30

for i, value in enumerate(rdd1.collect()):
    if top <= 0:
        break
    print(f"{i}| {value}")
    top-=1

genre_idx = [1,2,3,4,5,7,8,9,10,11,12,13,14,15,16]

0| ('국내도서', 44749)
1| ('인문', 7794)
2| ('대학교재', 7241)
3| ('경제/경영', 5368)
4| ('외국어', 4323)
5| ('소설', 3211)
6| ('청소년', 3074)
7| ('컴퓨터/IT', 2829)
8| ('시/에세이', 2815)
9| ('기술/공학', 2793)
10| ('예술/대중문화', 2783)
11| ('취미/실용/스포츠', 2562)
12| ('과학', 2415)
13| ('역사/문화', 2300)
14| ('정치/사회', 2277)
15| ('자기계발', 2221)
16| ('요리', 1879)
17| ('건강', 1725)
18| ('종교', 1638)
19| ('컴퓨터', 1458)
20| ('가정/육아', 1091)
21| ('교육학', 1079)
22| ('고전소설/문학선', 917)
23| ('기술공학', 887)
24| ('나라별에세이', 885)
25| ('인물/자전적에세이', 882)
26| ('장르소설', 874)
27| ('중/고등참고서', 863)
28| ('철학', 785)
29| ('인문학일반', 767)


In [14]:
top = 30

for i, value in enumerate(rdd2.collect()):
    if top <= 0:
        break
    print(f"{i}| {value}")
    top-=1


0| ('인간과사회', 1)
1| ('경찰기타과목', 1)
2| ('여성정책/제도', 1)
3| ('아시아정치', 1)
4| ('내과일반', 1)
5| ('노년에세이', 1)
6| ('어린이성경책', 1)
7| ('대한성서공회', 1)
8| ('음악사일반', 1)
9| ('라흐마니노프', 1)
10| ('의학윤리', 1)
11| ('프로엔지니어링', 1)
12| ('성악/합창이론', 1)
13| ('포스트모더니즘', 1)
14| ('액션/무협만화', 1)
15| ('식품학사전', 1)
16| ('중국근대사', 1)
17| ('PD/기자/아나운서', 1)
18| ('케라스/텐서플로', 1)
19| ('한국어학습일반', 1)
20| ('공무원교육학', 1)
21| ('장보드리야르', 1)
22| ('유가/유교', 1)
23| ('자기계발/공부방법/진로', 1)
24| ('.NET일반', 1)
25| ('중국작가론', 1)
26| ('공무원헌법', 1)
27| ('순자', 1)
28| ('친일문학론', 1)
29| ('발생생물학', 1)


## pracitice

In [4]:
temp = sc.parallelize([('이재정', '박희윤'), ("김민형", "이재정"), ("이재정", "박희윤")])
temp.take(1)
temp1 = temp.distinct()
for i in temp1.collect():
    print(i)


박희윤
김민형
이상원
이재정


In [34]:
s = set({('이재정', '박희윤')})
x = ('이재정', '박희윤')
print(s)
print(x)
if x in s:
    print("already")
else:
    print("not in set")

{('이재정', '박희윤')}
('이재정', '박희윤')
already


In [25]:
x = "asdfstringasdf"
y = "stirng"
if y in x:
    print("true")
else:
    print("false")

false
